In [1]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [13]:
rectangle = ee.Geometry.Rectangle([100.20, 37.96, 100.23, 37.99])  # ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)
roi = ee.Geometry(rectangle, None, False)
Map.addLayer(roi, {}, 'rectangle')

In [3]:
from Monarch import user_gee

In [11]:
images = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(roi)\
            .filterDate("2021-06", "2021-07").sort('CLOUD_COVER', False)\
            .map(user_gee.cloud_free_landsat_sr).mosaic()
visParams = {
  "bands": ['B4', 'B3', 'B2'],
  "min": 0,
  "max": 3000,
}

In [12]:
Map.addLayer(images, visParams, 'landsat')

In [15]:
geemap.ee_export_image(images, filename=r"H:/Monarch/image/landsat.tif", scale=30, region=roi)

Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat.tif


In [ ]:
clip_dow_merge(roi, temp_mean.subtract(273), r'E:/Data/temp/mean1', 8000)